In [1]:
#import all the required libraries
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm 
import cv2
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import keras 
import tensorflow
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, GaussianNoise, Conv2D, MaxPooling2D, UpSampling2D, Add, Input, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from keras.preprocessing.image import img_to_array
from tensorflow.keras.callbacks import EarlyStopping
import re
from skimage.metrics import structural_similarity
tensorflow.random.set_seed(0)

In [2]:
# set the path for the image directories
# for using in google colab please upload the data given along with the notebook to colab.
# after uploading please change the data variable to appropriate directory.
data = "../input/dataset1/train"

photos = data+"/photos"
sketches = data+"/sketches"

print(os.listdir(photos)[0])
print(os.listdir(sketches)[0])

In [3]:
# sort the photos with their names so that the right images match each other.
# these two lists store the paths of images and sketches which are ordered.
photo_paths_sorted = []
sketches_paths_sorted = []

# sortFiles function sorts the files.
def sortFiles(path1, path2):
  list_of_photos = sorted( filter( lambda x: os.path.isfile(os.path.join(path1, x)),
                        os.listdir(path1) ) )
  
  list_of_sketches = sorted( filter( lambda x: os.path.isfile(os.path.join(path2, x)),
                        os.listdir(path2) ) )
  for file_name in list_of_photos:
    #print(file_name)
    photo_paths_sorted.append(file_name)

  for file_name in list_of_sketches:
    #print(file_name)
    sketches_paths_sorted.append(file_name)

sortFiles(photos, sketches)

In [4]:
# this cell augments the data.
# this function augements the images, converts and stores them in arrays.
def photoarray(paths):
  array = []
    #this size will be used to reshape the images to SIZE dimensions.
  SIZE = 256
    #for each image perform the below transformations
  for i in paths:
    # read the images
    image = cv2.imread(photos+"/"+i,1)
    # convert them to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # resize the images
    image = cv2.resize(image, (SIZE, SIZE))
    # normalize the data
    image = image.astype('float32') / 255.0
    # convert the images to arrays. 
    array.append(img_to_array(image))
    # flip the image
    img1 = cv2.flip(image,1)
    # add to array
    array.append(img_to_array(img1))
    # flip the image
    img2 = cv2.flip(image,-1)
    # add to the array
    array.append(img_to_array(img2))
    img3 = cv2.flip(image,-1)
    img3 = cv2.flip(img3,1)
    array.append(img_to_array(img3))
    # rotate the images clockwise
    img4 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    # add to array
    array.append(img_to_array(img4))
    img5 = cv2.flip(img4,1)
    array.append(img_to_array(img5))
    # rotate the images counterclockwise
    img6 = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    # add to array
    array.append(img_to_array(img6))
    img7 = cv2.flip(img6,1)
    array.append(img_to_array(img7))
    
    # return the array
  return array

# same transformations are then performed for sketches.
def sketcharray(paths):
  array = []
  SIZE = 256
  for i in paths:
    image = cv2.imread(sketches+"/"+i,1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (SIZE, SIZE))
    image = image.astype('float32') / 255.0
    array.append(img_to_array(image))
    img1 = cv2.flip(image,1)
    array.append(img_to_array(img1))
    img2 = cv2.flip(image,-1)
    array.append(img_to_array(img2))
    img3 = cv2.flip(image,-1)
    img3 = cv2.flip(img3,1)
    array.append(img_to_array(img3))
    img4 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    array.append(img_to_array(img4))
    img5 = cv2.flip(img4,1)
    array.append(img_to_array(img5))
    img6 = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    array.append(img_to_array(img6))
    img7 = cv2.flip(img6,1)
    array.append(img_to_array(img7))

  return array

# this functions makes sure right paths are provided to convert the images and sketches to arrays
def covertToArrays(photo_paths, sketches_path):
  image_arrays = photoarray(photo_paths)
  sketch_arrays = sketcharray(sketches_path)

  return image_arrays, sketch_arrays

image_arrays, sketch_arrays = covertToArrays(photo_paths_sorted, sketches_paths_sorted)

In [5]:
# print out the final number of images.
print("Total number of sketch images:",len(sketch_arrays))
print("Total number of images:",len(image_arrays))

In [6]:
# defining function to plot images pair
def plot_images(image, sketches):
    plt.figure(figsize=(7,7))
    plt.subplot(1,2,1)
    plt.title('Image', color = 'green', fontsize = 20)
    plt.imshow(image)
    plt.subplot(1,2,2)
    plt.title('Sketches ', color = 'black', fontsize = 20)
    plt.imshow(sketches)
   
    plt.show()

In [7]:
# displaying images
ls = [i for i in range(0,65,8)]
for i in ls:
    plot_images(image_arrays[i],sketch_arrays[i])

In [8]:
# this cell splits the arrays obtained into training and testing dataset.
SIZE = 256
# 90% of data are assigned to training
train_sketch_image = sketch_arrays[:630]
train_image = image_arrays[:630]
# 10% of data are assigned to testing
test_sketch_image = sketch_arrays[630:]
test_image = image_arrays[630:]
# reshaping
train_sketch_image = np.reshape(train_sketch_image,(len(train_sketch_image),SIZE,SIZE,3))
train_image = np.reshape(train_image, (len(train_image),SIZE,SIZE,3))
print('Train color image shape:',train_image.shape)
test_sketch_image = np.reshape(test_sketch_image,(len(test_sketch_image),SIZE,SIZE,3))
test_image = np.reshape(test_image, (len(test_image),SIZE,SIZE,3))
print('Test color image shape',test_image.shape)

# First Model

In [9]:
# an early stopping with patience 10 is declared. This is used to avoid overfitting.
early_stop = EarlyStopping(monitor="val_loss, patience=10")

In [10]:
# this is the first model 
# an encoder is created.
encoder = Sequential()
encoder.add(Flatten(input_shape=[256,256,3]))
encoder.add(Dense(64, activation="relu"))
encoder.add(Dense(32, activation="relu"))
encoder.add(Dense(16, activation="relu"))

# a decoder is declared.
decoder = Sequential()
decoder.add(Dense(32, input_shape=[16], activation="relu"))
decoder.add(Dense(64, activation="relu"))
decoder.add(Dense(256*256*3, activation="sigmoid"))
decoder.add(Reshape([256,256,3]))

# both the encoder and decoder are combined to form a autoencoder.
autoencoder = Sequential([encoder, decoder])

# the model is compiled. 
autoencoder.compile(loss="mean_absolute_error",
                    optimizer="adam",
                    metrics=["accuracy"])

autoencoder.summary()

In [11]:
# this function is used to visualize the model.
tf.keras.utils.plot_model(autoencoder, to_file="model_1.png", show_shapes=True)

## Model 1 face2sketch

In [12]:
# the model is fit.
hist = autoencoder.fit(train_image, train_sketch_image, epochs=200, verbose=0, validation_data=(test_image, test_sketch_image), callbacks=[early_stop])

# model is evaluated for checking the accuracy. 
prediction_on_test_data = autoencoder.evaluate(test_image, test_sketch_image)
print("Loss: ", prediction_on_test_data[0])
print("Accuracy: ", np.round(prediction_on_test_data[1] * 100,1))

# predictions are made on the test data
pred=autoencoder.predict(test_image)

# results are displayed on the test data.
n =10
plt.imshow(test_image[n])
plt.show()
plt.imshow(test_sketch_image[n])
plt.show()
plt.imshow(pred[n].reshape(256,256,3))

In [13]:
# visualize the loss of the model during taining and testing.
metrics = pd.DataFrame(hist.history)
metrics[["loss", "val_loss"]].plot()

In [14]:
# this cell is used to calculate the structural similarity of between original and predicted results.
imageA = test_image[18]
imageB = pred[18]

# Convert the images to grayscale
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

# Compute the Structural Similarity Index (SSIM) between the two
# images, ensuring that the difference image is returned
(score, diff) = structural_similarity(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")

# You can print only the score if you want
print("SSIM: {}".format(score))

## Model 1 sketch2face

In [15]:
# the model is fit.
hist1 = autoencoder.fit(train_sketch_image, train_image, epochs=200, verbose=0, validation_data=(test_sketch_image, test_image), callbacks=[early_stop])

# model is evaluated for checking the accuracy. 
prediction_on_test_data = autoencoder.evaluate(test_sketch_image, test_image)
print("Loss: ", prediction_on_test_data[0])
print("Accuracy: ", np.round(prediction_on_test_data[1] * 100,1))

# predictions are made on the test data
pred=autoencoder.predict(test_sketch_image)

# results are displayed on the test data.
n =10
plt.imshow(test_image[n])
plt.show()
plt.imshow(test_sketch_image[n])
plt.show()
plt.imshow(pred[n].reshape(256,256,3))

In [16]:
# visualize the loss of the model during taining and testing.
metrics = pd.DataFrame(hist.history)
metrics[["loss", "val_loss"]].plot()

In [17]:
# this cell is used to calculate the structural similarity of between original and predicted results.
imageA = test_sketch_image[18]
imageB = pred[18]

# Convert the images to grayscale
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

# Compute the Structural Similarity Index (SSIM) between the two
# images, ensuring that the difference image is returned
(score, diff) = structural_similarity(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")

# You can print only the score if you want
print("SSIM: {}".format(score))

# Second Model

In [18]:
# this is the second model which is a simple convolution autoencoder.
model=Sequential()
# input layer
model.add(Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(256,256,3)))
# pooling layer
model.add(MaxPooling2D((2,2), padding='same'))
# conv layer
model.add(Conv2D(32, (3,3),activation='relu',padding='same'))
# pooling layer
model.add(MaxPooling2D((2,2), padding='same'))
# conv layer
model.add(Conv2D(16, (3,3),activation='relu',padding='same'))
# pooling layer
model.add(MaxPooling2D((2,2), padding='same'))
# conv layer decoder side
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
# upscaling layer
model.add(UpSampling2D((2,2)))
# conv layer
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
# upscaling layer
model.add(UpSampling2D((2,2)))
# conv layer
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
# upscaling layer
model.add(UpSampling2D((2,2)))
# conv layer
model.add(Conv2D(3, (3,3), activation='relu', padding='same'))

# model is compiled
model.compile(optimizer="adam",loss='mean_squared_error',metrics=['accuracy'])
# model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

In [19]:
# this function is used to visualize the model.
tf.keras.utils.plot_model(model, to_file="model_2.png", show_shapes=True)

## Model 2 face2sketch

In [20]:
# model is fit
hist = model.fit(train_image, train_sketch_image, epochs=200, shuffle=True, verbose=0, validation_data=(test_image, test_sketch_image), callbacks=[early_stop])

prediction_on_test_data = model.evaluate(test_image, test_sketch_image)
print("Loss: ", prediction_on_test_data[0])
print("Accuracy: ", np.round(prediction_on_test_data[1] * 100,1))

# predictions are made.
pred=model.predict(test_image)

# results are visualized
n = 10
plt.imshow(test_image[n])
plt.show()
plt.imshow(test_sketch_image[n])
plt.show()
plt.imshow(pred[n].reshape(256,256,3))

In [21]:
# visualize the loss of the model during taining and testing.
metrics = pd.DataFrame(hist.history)
metrics[["loss", "val_loss"]].plot()

In [22]:
# this cell is used to calculate the structural similarity of between original and predicted results.
imageA = test_image[18]
imageB = pred[18]

# Convert the images to grayscale
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

# Compute the Structural Similarity Index (SSIM) between the two
# images, ensuring that the difference image is returned
(score, diff) = structural_similarity(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")

# You can print only the score if you want
print("SSIM: {}".format(score))

## Model 2 sketch2face

In [23]:
# model is fit
hist = model.fit(train_sketch_image, train_image, epochs=100, shuffle=True, verbose=0, validation_data=(test_sketch_image, test_image), callbacks=[early_stop])

prediction_on_test_data = model.evaluate(test_sketch_image, test_image)
print("Loss: ", prediction_on_test_data[0])
print("Accuracy: ", np.round(prediction_on_test_data[1] * 100,1))

# predictions are made on testing data
pred=model.predict(test_sketch_image)

# results are visualized
n = 10
plt.imshow(test_image[n])
plt.show()
plt.imshow(test_sketch_image[n])
plt.show()
plt.imshow(pred[n].reshape(256,256,3))

In [24]:
# visualize the loss of the model during taining and testing.
metrics = pd.DataFrame(hist.history)
metrics[["loss", "val_loss"]].plot()

In [25]:
imageA = test_sketch_image[18]
imageB = pred[18]

# Convert the images to grayscale
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

# Compute the Structural Similarity Index (SSIM) between the two
# images, ensuring that the difference image is returned
(score, diff) = structural_similarity(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")

# You can print only the score if you want
print("SSIM: {}".format(score))

# Third Model

In [26]:
# the third model is also a convolution autoencoder but with a but if complexity
# the downsampled function is defined. defining the encoder side of autoencoder.
def downsample(filters, size, apply_batch_normalization = True):
    downsample = Sequential()
    # a conv layer
    downsample.add(tensorflow.keras.layers.Conv2D(filters = filters, kernel_size = size, strides = 2, use_bias = False, kernel_initializer = 'he_normal'))
    # provisions to apply batch normalization
    if apply_batch_normalization:
        downsample.add(tensorflow.keras.layers.BatchNormalization())
    downsample.add(tensorflow.keras.layers.LeakyReLU())
    return downsample

# the upsample function is defined. defining the decoder side of autoencoder.
def upsample(filters, size, apply_dropout = False):
    upsample = Sequential()
    # a conv layer
    upsample.add(tensorflow.keras.layers.Conv2DTranspose(filters = filters, kernel_size = size, strides = 2, use_bias = False, kernel_initializer = 'he_normal'))
    if apply_dropout:
        # provisions to apply dropout
        upsample.add(tensorflow.keras.layers.Dropout(0.1))
    upsample.add(tensorflow.keras.layers.LeakyReLU()) 
    return upsample

# the model is defined here
def model():
    # input layer
    encoder_input = tensorflow.keras.Input(shape = (SIZE, SIZE, 3))
    # layers of encoder.
    x = downsample(16, 4, False)(encoder_input)
    x = downsample(32,4)(x)
    x = downsample(64,4,True)(x)
    x = downsample(128,4)(x)
    x = downsample(256,4)(x)
   
    encoder_output = downsample(512,4)(x)
    
    decoder_input = upsample(512,4,True)(encoder_output)
    # layers of decoder
    x = upsample(256,4,False)(decoder_input)
    x = upsample(128,4, True)(x)
    x = upsample(64,4)(x)
    x = upsample(32,4)(x)
    x = upsample(16,4)(x)
    x = tensorflow.keras.layers.Conv2DTranspose(8,(2,2),strides = (1,1), padding = 'valid')(x)
    decoder_output = tensorflow.keras.layers.Conv2DTranspose(3,(2,2),strides = (1,1), padding = 'valid')(x)
    
    # the encoder and decoder is combined to make the autoencoder
    return tensorflow.keras.Model(encoder_input, decoder_output)


model = model()
model.summary()
# model is compiled
model.compile(optimizer = tensorflow.keras.optimizers.Adam(learning_rate = 0.001), loss = 'mean_absolute_error',
              metrics = ['acc'])

In [27]:
tf.keras.utils.plot_model(model, to_file="model_3.png", show_shapes=True)

## Model 3 face2sketch

In [28]:
# model is fit
hist = model.fit(train_image, train_sketch_image, epochs=200, shuffle=True, verbose=0, validation_data=(test_image, test_sketch_image), callbacks=[early_stop])

prediction_on_test_data = model.evaluate(test_image, test_sketch_image)
print("Loss: ", prediction_on_test_data[0])
print("Accuracy: ", np.round(prediction_on_test_data[1] * 100,1))

# predictions are made on testing data
pred=model.predict(test_image)

In [29]:
n = 18
plt.imshow(test_image[n])
plt.show()
plt.imshow(test_sketch_image[n])
plt.show()
plt.imshow(pred[n].reshape(256,256,3))

In [30]:
# loss is visualized
metrics = pd.DataFrame(hist.history)
metrics[["loss", "val_loss"]].plot()

In [31]:
imageA = test_image[18]
imageB = pred[18]

# Convert the images to grayscale
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

# Compute the Structural Similarity Index (SSIM) between the two
# images, ensuring that the difference image is returned
(score, diff) = structural_similarity(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")

# You can print only the score if you want
print("SSIM: {}".format(score))

## Model 3 sketch2face

In [32]:
# model is fit
hist = model.fit(train_sketch_image, train_image, epochs=200, shuffle=True, verbose=0, validation_data=(test_sketch_image, test_image), callbacks=[early_stop])

prediction_on_test_data = model.evaluate(test_sketch_image, test_image)
print("Loss: ", prediction_on_test_data[0])
print("Accuracy: ", np.round(prediction_on_test_data[1] * 100,1))

# predictions are made
pred=model.predict(test_sketch_image)

In [33]:
n = 18
plt.imshow(test_image[n])
plt.show()
plt.imshow(test_sketch_image[n])
plt.show()
plt.imshow(pred[n].reshape(256,256,3))

In [34]:
# loss is visualized
metrics = pd.DataFrame(hist.history)
metrics[["loss", "val_loss"]].plot()

In [35]:
imageA = test_sketch_image[18]
imageB = pred[18]

# Convert the images to grayscale
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

# Compute the Structural Similarity Index (SSIM) between the two
# images, ensuring that the difference image is returned
(score, diff) = structural_similarity(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")

# You can print only the score if you want
print("SSIM: {}".format(score))

# Fourth Model

In [36]:
model1=Sequential()

model1.add(Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(256,256,3)))
model1.add(MaxPooling2D((2,2), padding='same'))
model1.add(Conv2D(32, (3,3),activation='relu',padding='same'))
model1.add(MaxPooling2D((2,2), padding='same'))
model1.add(Conv2D(16, (3,3),activation='relu',padding='same'))
model1.add(MaxPooling2D((2,2), padding='same'))

model1.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model1.add(UpSampling2D((2,2)))

model1.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model1.add(UpSampling2D((2,2)))

model1.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model1.add(UpSampling2D((2,2)))

model1.add(Conv2D(3, (3,3), activation='relu', padding='same'))

def downsample(filters, size, apply_batch_normalization = True):
    downsample = Sequential()
    downsample.add(tensorflow.keras.layers.Conv2D(filters = filters, kernel_size = size, strides = 2, use_bias = False, kernel_initializer = 'he_normal'))
    if apply_batch_normalization:
        downsample.add(tensorflow.keras.layers.BatchNormalization())
    downsample.add(tensorflow.keras.layers.LeakyReLU())
    return downsample

def upsample(filters, size, apply_dropout = False):
    upsample = Sequential()
    upsample.add(tensorflow.keras.layers.Conv2DTranspose(filters = filters, kernel_size = size, strides = 2, use_bias = False, kernel_initializer = 'he_normal'))
    if apply_dropout:
        upsample.add(tensorflow.keras.layers.Dropout(0.1))
    upsample.add(tensorflow.keras.layers.LeakyReLU()) 
    return upsample

def model2():
    encoder_input = tensorflow.keras.Input(shape = (SIZE, SIZE, 3))
    x = downsample(16, 4, False)(encoder_input)
    x = downsample(32,4)(x)
    x = downsample(64,4,False)(x)
    x = downsample(128,4)(x)
    x = downsample(256,4)(x)
   
    encoder_output = downsample(512,4)(x)
    
    decoder_input = upsample(512,4,True)(encoder_output)
    x = upsample(256,4,False)(decoder_input)
    x = upsample(128,4, True)(x)
    x = upsample(64,4)(x)
    x = upsample(32,4)(x)
    x = upsample(16,4)(x)
    x = tensorflow.keras.layers.Conv2DTranspose(8,(2,2),strides = (1,1), padding = 'valid')(x)
    decoder_output = tensorflow.keras.layers.Conv2DTranspose(3,(2,2),strides = (1,1), padding = 'valid')(x)
    
  
    return tensorflow.keras.Model(encoder_input, decoder_output)

model2 = model2()

encoder = Sequential()
encoder.add(Flatten(input_shape=[256,256,3]))
encoder.add(Dense(64, activation="relu"))
encoder.add(Dense(32, activation="relu"))
encoder.add(Dense(16, activation="relu"))

decoder = Sequential()
decoder.add(Dense(32, input_shape=[16], activation="relu"))
decoder.add(Dense(64, activation="relu"))
decoder.add(Dense(256*256*3, activation="sigmoid"))
decoder.add(Reshape([256,256,3]))

model3 = Sequential([encoder, decoder])

# a common input is given to all models.
commonInput = Input(shape=(256,256,3))

# three models are given the same imput.
out1 = model1(commonInput)    
out2 = model2(commonInput)  
out3 = model3(commonInput)

# the three models are added together
mergedOut = Add()([out1,out2,out3])

# a final ensemble model is created.
oneInputModel = Model(commonInput,mergedOut)

# ensemble model is compiled.
oneInputModel.compile(optimizer="adam",loss='mean_absolute_error',metrics=['accuracy'])
oneInputModel.summary()

In [37]:
tf.keras.utils.plot_model(model, to_file="model_4.png", show_shapes=True)

## Model 4 sketch2face

In [38]:
# model is fit
hist = oneInputModel.fit(train_sketch_image, train_image, epochs=200, shuffle=True, verbose=0, validation_data=(test_sketch_image, test_image), callbacks=[early_stop])

prediction_on_test_data = oneInputModel.evaluate(test_sketch_image, test_image)
print("Loss: ", prediction_on_test_data[0])
print("Accuracy: ", np.round(prediction_on_test_data[1] * 100,1))

# predictions are made.
pred=oneInputModel.predict(test_sketch_image)

In [39]:
# results are visualized
n=18
plt.imshow(test_image[n])
plt.show()
plt.imshow(test_sketch_image[n])
plt.show()
plt.imshow(pred[n].reshape(256,256,3))

In [40]:
# loss is visualized
metrics = pd.DataFrame(hist.history)
metrics[["loss", "val_loss"]].plot()

In [41]:
imageA = test_image[18]
imageB = pred[18]

# 4. Convert the images to grayscale
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

# 5. Compute the Structural Similarity Index (SSIM) between the two
#    images, ensuring that the difference image is returned
(score, diff) = structural_similarity(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")

# 6. You can print only the score if you want
print("SSIM: {}".format(score))

## Mode 4 face2sketch

In [42]:
# model is fit
hist = oneInputModel.fit(train_image,train_sketch_image, epochs=200, shuffle=True, verbose=0, validation_data=(test_image, test_sketch_image), callbacks=[early_stop])

prediction_on_test_data = oneInputModel.evaluate(test_image, test_sketch_image)
print("Loss: ", prediction_on_test_data[0])
print("Accuracy: ", np.round(prediction_on_test_data[1] * 100,1))

# predictions are made.
pred=oneInputModel.predict(test_image)

In [43]:
# results are visualized
n=18
plt.imshow(test_image[n])
plt.show()
plt.imshow(test_sketch_image[n])
plt.show()
plt.imshow(pred[n].reshape(256,256,3))

In [44]:
# loss is visualized
metrics = pd.DataFrame(hist.history)
metrics[["loss", "val_loss"]].plot()

In [45]:
imageA = test_image[18]
imageB = pred[18]

# 4. Convert the images to grayscale
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

# 5. Compute the Structural Similarity Index (SSIM) between the two
#    images, ensuring that the difference image is returned
(score, diff) = structural_similarity(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")

# 6. You can print only the score if you want
print("SSIM: {}".format(score))